In [ ]:
# GTM Package including GTM and the plot functions
from gtm import *

import os
from torch.utils.data import DataLoader

from demos.load_magic_data_helpers import *
from demos.dataset_helpers import Generic_Dataset

/Users/matthiasherp/anaconda3/envs/mctm_pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# got up one directory as new working directory so that loading data works
# only run this once
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
print(os.getcwd())

/Users/matthiasherp/Desktop/phd_github_repositories/gtm


#### Load Data

In [ ]:
group = "h"

y_train_group_h, y_validate_group_h, y_test_group_h = load_magic_data(group=group,
                                                            train_portion=2/3,
                                                            data_dims=10,
                                                            poly_span_abs=12,
                                                            cross_validation_folds=5,
                                                            validation_fold_index=4,
                                                            split_random_state=25)
                

In [ ]:
group = "g"

y_train_group_g, y_validate_group_g, y_test_group_g = load_magic_data(group=group,
                                                            train_portion=2/3,
                                                            data_dims=10,
                                                            poly_span_abs=12,
                                                            cross_validation_folds=5,
                                                            validation_fold_index=4,
                                                            split_random_state=25)
                

## Classification of Test Set